In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
datasets = [
    ["abalone_input.pd","abalone_target.pd"], 
    ["adult_input.pd","adult_target.pd"], # Large
    ["banknote_input.pd","banknote_target.pd"], # Easy
    ["bank_input.pd","bank_target.pd"], # Large
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["cars_input.pd","cars_target.pd"], 
    ["contraceptive_input.pd","contraceptive_target.pd"], 
    ["generated6_input.np","generated6_target.np"],
    ["hrss_input.pd","hrss_target.pd"], # Large
    ["iris_input.pd","iris_target.pd"],
    ["steel_input.pd","steel_target.pd"],
    ["students_input.pd","students_target.pd"],
    #["sensorless_input.pd","sensorless_target.pd"], # Very Large dataset
]

In [3]:
gate_reduction_methods = [
    "feature_importance",
    "feature_importance_lda",
    "feature_importance_lda_max",
    "feature_importance_lr",
    "feature_importance_lr_max",
    "feature_importance_xgb",
    "feature_importance_pca_loadings",
    "PCA",
    None
]

In [4]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": 3,
    "iterations": 100,
    "max_depth": 1,
    "init_learning_rate": 110,
    "learning_rate_decay": 0.985,
    "initialization_method": "random",
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": "overwritten",
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    "verbose": False
    }
parameters_fit = {
    "optimization_method": "least_squares_linear_regression",
    "early_stopping": "likelihood",
    }

In [5]:
start = timer()
runs = 50
rows = []
for dataset in datasets:
    print("Starting",dataset[0],"...")
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    
    use_dataframe = False
    if isinstance(data_input, pd.core.frame.DataFrame):
        use_dataframe = True
        
      
    dimensionality_reduction = gate_reduction_methods
    dict_results = {
        "dataset" : dataset[0],
        "n_features" : data_input.shape[1]
    } 
    
    for method in dimensionality_reduction:
        print("Starting",method,"...")
        parameters["use_2_dim_gate_based_on"] = method

        train_accuracies = []
        val_accuracies = []
        rkf = RepeatedKFold(n_splits=4, n_repeats=runs)
        for train_idx, val_idx in rkf.split(data_input):
            if use_dataframe:
                X_temp = data_input.iloc[train_idx]
                y_temp = data_target.iloc[train_idx]
                X_temp.reset_index(inplace=True, drop=True)
                y_temp.reset_index(inplace=True, drop=True)
            else:
                X_temp = data_input[train_idx]
                y_temp = data_target[train_idx]

            parameters["X"] = X_temp
            parameters["y"] = y_temp
            parameters["initialization_method"] = Random_init(seed=None)
            modt = MoDT(**parameters)
            modt.fit(**parameters_fit)
            train_accuracies.append(modt.score_internal_disjoint())

            if use_dataframe:
                X_temp = data_input.iloc[val_idx]
                y_temp = data_target.iloc[val_idx]
                X_temp.reset_index(inplace=True, drop=True)
                y_temp.reset_index(inplace=True, drop=True)
            else:
                X_temp = data_input[val_idx]
                y_temp = data_target[val_idx]
            val_accuracies.append(modt.score(X_temp, y_temp))

        train_accuracy = np.mean(train_accuracies)
        val_accuracy = np.mean(val_accuracies)
        train_std = np.std(train_accuracies)
        val_std = np.std(val_accuracies)
        dict_results[str(method) + "_train"] = train_accuracy
        dict_results[str(method) + "_test"] = val_accuracy
        dict_results[str(method) + "_train_std"] = train_std
        dict_results[str(method) + "_test_std"] = val_std
        
    rows.append(dict_results)
    
print("Duration", timer() - start)
df_performance = pd.DataFrame(rows)

Starting abalone_input.pd ...
Starting feature_importance ...
Starting feature_importance_lda ...
Starting feature_importance_lda_max ...
Starting feature_importance_lr ...
Starting feature_importance_lr_max ...
Starting feature_importance_xgb ...
Starting feature_importance_pca_loadings ...
Starting PCA ...
Starting None ...
Starting adult_input.pd ...
Starting feature_importance ...
Starting feature_importance_lda ...
Starting feature_importance_lda_max ...
Starting feature_importance_lr ...
Starting feature_importance_lr_max ...
Starting feature_importance_xgb ...
Starting feature_importance_pca_loadings ...
Starting PCA ...
Starting None ...
Starting banknote_input.pd ...
Starting feature_importance ...
Starting feature_importance_lda ...
Starting feature_importance_lda_max ...
Starting feature_importance_lr ...
Starting feature_importance_lr_max ...
Starting feature_importance_xgb ...
Starting feature_importance_pca_loadings ...
Starting PCA ...
Starting None ...
Starting bank_inp

In [6]:
df_performance

,dataset,n_features,feature_importance_train,feature_importance_test,feature_importance_train_std,feature_importance_test_std,feature_importance_lda_train,feature_importance_lda_test,feature_importance_lda_train_std,feature_importance_lda_test_std,...,feature_importance_pca_loadings_train_std,feature_importance_pca_loadings_test_std,PCA_train,PCA_test,PCA_train_std,PCA_test_std,None_train,None_test,None_train_std,None_test_std
0,abalone_input.pd,8,0.716875,0.702687,0.009870,0.015337,0.733269,0.724046,0.009698,0.016813,...,0.007853,0.014592,0.683495,0.671783,0.010457,0.016736,0.746287,0.733105,0.009140,0.013700
1,adult_input.pd,14,0.786720,0.785077,0.028776,0.029480,0.831814,0.831384,0.003941,0.008273,...,0.021588,0.021671,0.820786,0.810455,0.012798,0.017152,0.848417,0.777455,0.005526,0.094549
2,banknote_input.pd,4,0.954670,0.946283,0.027148,0.029202,0.998406,0.994927,0.001904,0.003465,...,0.004495,0.014577,0.955389,0.947055,0.014724,0.018051,0.999883,0.998367,0.000358,0.002666
3,bank_input.pd,20,0.904894,0.903594,0.002427,0.003352,0.898421,0.897604,0.007051,0.007754,...,0.001342,0.003093,0.895609,0.892794,0.005856,0.007138,0.910126,0.880354,0.004962,0.051504
4,breast_cancer_input.np,10,0.954106,0.925099,0.009658,0.022624,0.932197,0.899395,0.010336,0.022240,...,0.012427,0.022379,0.940657,0.913780,0.009563,0.022547,0.975946,0.928018,0.008633,0.022314
5,cars_input.pd,6,0.784232,0.782859,0.013974,0.019641,0.784757,0.782396,0.014134,0.017874,...,0.031920,0.035711,0.767141,0.757488,0.026974,0.030035,0.910517,0.894167,0.012916,0.017239
6,contraceptive_input.pd,9,0.567979,0.553156,0.018690,0.030707,0.568491,0.554232,0.016734,0.029284,...,0.006921,0.022067,0.499963,0.475817,0.013948,0.025099,0.576986,0.511657,0.023949,0.029690
7,generated6_input.np,2,0.859785,0.856276,0.056531,0.056651,0.858433,0.855296,0.057093,0.057544,...,0.072125,0.072116,0.855981,0.852800,0.054406,0.055476,0.855827,0.851540,0.055034,0.056628
8,hrss_input.pd,18,0.764688,0.764392,0.002276,0.005133,0.764832,0.764733,0.002216,0.005111,...,0.002620,0.005507,0.764947,0.764600,0.002357,0.004840,0.767292,0.766613,0.003150,0.005488
9,iris_input.pd,4,0.982268,0.950814,0.011503,0.034600,0.980844,0.953321,0.011692,0.037688,...,0.017740,0.039255,0.972135,0.937351,0.009518,0.032916,0.991827,0.946682,0.008526,0.036405


In [7]:
pickle.dump(df_performance, open("dataframes/ex4b_df_selection_performances.pd", "wb"))